In [1]:
import pyspark

In [2]:
pyspark.__file__

'/home/abyssde232024/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py'

In [3]:
from pyspark.sql import SparkSession

In [4]:
print(SparkSession.getActiveSession())

None


In [5]:
spark_sql = SparkSession.builder \
    .master("local[*]") \
    .appName('dezoomcamp2024') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/02 17:13:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df_green = spark_sql.read.parquet('data/processed/green/*/*', header=True)

In [9]:
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-26 12:45:44|  2020-01-26 12:45:45|                 N|         5|          25|         264|              1|          .00|      40.05|    0|      

In [12]:
df_green.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



In [10]:
df_yellow = spark_sql.read.parquet('data/processed/yellow/*/*', header=True)

In [11]:
df_yellow.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       2| 2020-01-06 09:18:38|  2020-01-06 09:33:56|              1|         3.03|         1|                 N|         263|         233|           1|       12.0|  0.0|    0.5|       1.5|         0.0|                  0.3

In [13]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [16]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [17]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

set instruction is to check intersection of values between two set

In [14]:
set(df_green.columns) - set(df_yellow.columns)  # {'ehail_fee', 'trip_type', 'total_amount'}

{'ehail_fee', 'lpep_dropoff_datetime', 'lpep_pickup_datetime', 'trip_type'}

In [15]:
set(df_yellow.columns) - set(df_green.columns)  # {'store_and_fwd_flag', 'rate_code', 'fare_amount'}

{'tpep_dropoff_datetime', 'tpep_pickup_datetime'}

Rename dataframe column names to essentially having same name in both df

In [24]:
df_yellow = df_yellow \
            .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime') \
            .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')

In [25]:
df_green = df_green \
            .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime') \
            .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')

In [26]:
df_green.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [27]:
df_yellow.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

Recreate the columns in the yellow taxi data so that it has the same columns as the green taxi data. Fill in the missing columns with null values.

In [30]:
#  create a common column list which is present in both green and yellow dataframes and order them same as green dataframe
common_columns = []
yellow_columns = df_yellow.columns
for column in df_green.columns:
    if column in yellow_columns:
        common_columns.append(column)

In [32]:
common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [34]:
df_green.select(common_columns).show(5)

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+
|       2|2020-01-26 12:45:44|2020-01-26 12:45:45|                 N|         5|          25|         264|              1|          .00|      40.05|    0|      0|      8.07|           0|                  0.3|       48.42|           

In [36]:
# add service type column to green dataframe and yellow dataframe
from pyspark.sql import functions as F
df_green = df_green.withColumn('service_type', F.lit('green'))    

In [37]:
df_yellow = df_yellow.withColumn('service_type', F.lit('yellow'))

In [42]:
df_yellow.show(5)
df_yellow_select = df_yellow.select(common_columns + ['service_type'])

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|service_type|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------+
|       2|2020-01-06 09:18:38|2020-01-06 09:33:56|              1|         3.03|         1|                 N|         263|         233|           1|       12.0|  0.0|    0.5|       1.5|       

In [43]:
df_green.show(5)
df_green_select = df_green.select(common_columns + ['service_type'])

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+------------+
|       2|2020-01-26 12:45:44|2020-01-26 12:45:45|                 N|         5|          25|         264|              1|          .

In [45]:
# union both green and yellow dataframes
df_trip_data = df_green_select.unionAll(df_yellow_select)

In [46]:
# run group by on the dataframe to get the count of each service type
df_trip_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [47]:
# create temporary table from the dataframe
df_trip_data.registerTempTable('trip_data')

/home/abyssde232024/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [48]:
# use spark sql to get the count of each service type
spark_sql.sql('select service_type, count(1) as count from trip_data group by service_type').show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [53]:
df_result = spark_sql.sql('''
select 
    -- Reveneue grouping 
    PULocationID as revenue_zone,
    date_trunc("month", "pickup_datetime") as revenue_month, 
    service_type, 

    -- Revenue calculation 
    sum(fare_amount) as revenue_monthly_fare,
    sum(extra) as revenue_monthly_extra,
    sum(mta_tax) as revenue_monthly_mta_tax,
    sum(tip_amount) as revenue_monthly_tip_amount,
    sum(tolls_amount) as revenue_monthly_tolls_amount,
    sum(improvement_surcharge) as revenue_monthly_improvement_surcharge,
    sum(total_amount) as revenue_monthly_total_amount,

    -- Additional calculations
    avg(passenger_count) as avg_monthly_passenger_count,
    avg(trip_distance) as avg_monthly_trip_distance

from trip_data
group by 1,2,3
''')


In [54]:
df_result.printSchema()

root
 |-- revenue_zone: integer (nullable = true)
 |-- revenue_month: timestamp (nullable = true)
 |-- service_type: string (nullable = false)
 |-- revenue_monthly_fare: double (nullable = true)
 |-- revenue_monthly_extra: double (nullable = true)
 |-- revenue_monthly_mta_tax: double (nullable = true)
 |-- revenue_monthly_tip_amount: double (nullable = true)
 |-- revenue_monthly_tolls_amount: double (nullable = true)
 |-- revenue_monthly_improvement_surcharge: double (nullable = true)
 |-- revenue_monthly_total_amount: double (nullable = true)
 |-- avg_monthly_passenger_count: double (nullable = true)
 |-- avg_monthly_trip_distance: double (nullable = true)



In [57]:
df_result.show(5)

+------------+-------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|revenue_zone|revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|avg_monthly_passenger_count|avg_monthly_trip_distance|
+------------+-------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|         254|         null|       green|   278150.7099999999|   11320.519999999999|                  854.5|         9569.900000000001|       

In [56]:
df_result.repartition(4).write.parquet('data/report/revenue', mode='overwrite')

In [58]:
df_result.write.parquet('data/report/revenue', mode='overwrite')